# <font color='red'> Description du projet </font>

## <font color='blue'>Présentation du problème </font>

L’objectif de ce projet est d’estimer **les temps de réponse et de mobilisation** de la Brigade des Pompiers de Londres. La brigade des pompiers de Londres est le service d'incendie et de sauvetage le plus actif du Royaume-Uni  et l'une des plus grandes organisations de lutte contre l'incendie et de sauvetage au monde.

Le premier jeu de données fourni contient les détails de chaque incident traité depuis janvier 2009. Des informations sont fournies sur la date et le lieu de l'incident ainsi que sur le type d'incident traité. Il est composé de deux fichiers

*   LFB Incident data from 2009 - 2017.xlsx
*   LFB Incident data from 2018 onwards.csv

Le second fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-incident-records/f5066d66-c7a3-415f-9629-026fbda61822/LFB%20Incident%20data%20from%202018%20onwards.csv.xlsx' pour avoir la dernière version du fichier. En effet, les données sont mises à jour tous les mois. Il faut compter au moins 7 minutes pour la lecture des données.

<br>

Le second jeu de données contient les détails de chaque camion de pompiers envoyé sur les lieux d'un incident depuis janvier 2009. Des informations sont fournies sur l'appareil mobilisé, son lieu de déploiement et les heures d'arrivée sur les lieux de l'incident. Il est composé de trois fichiers

*   LFB Mobilisation data from January 2009 - 2014.xlsx
*   LFB Mobilisation data from 2015 - 2020.xlsx
*   LFB Mobilisation data from January 2009 - 2014.xlsx

Le dernier fichier peut-être récupéré à l'aide du lien : 'https://data.london.gov.uk/download/london-fire-brigade-mobilisation-records/3ff29fb5-3935-41b2-89f1-38571059237e/LFB%20Mobilisation%20data%202021%20-%202024.xlsx' pour avoir la dernière version du fichier (mise à jour mensuelle). Il faut compter environ 17 minutes pour la lecture des données.

## <font color='blue'> Etapes précédentes </font>



*   1 - Exploration des données : premières analyses, concaténation des différents fichiers puis jointure des 2 types de données (incident / mobilisation)
*   2 - Data visualisation.ipynb : visualisation des données, étude de la variable à prédire (temps de réponse total) en fonction des variables explicatives, création d'un jeu de données pour la modélisation

Dans le notebook *1 - Exploration des données*, nous avons crée un dataframe df_mobilisation_incident. Il est utilisé ici.

## <font color='blue'>Etapes dans ce notebook </font>

Au cours de la modélisation, nous avons été confronté à trois problématiques :
* le manque d'information sur certaines variables du jeu de données et la découverte tardive de certaines définitions implicant la requalification du caractère *a priori* de certaines d'entre elles
* la taille du jeu de données (plus d'un millions d'observations)
* le nombre important de variables explicatives une fois binarisées

Ce notebook correspond à une étape de notre modélisation où nous avions des informations incomplètes sur la définition de la variable `DetailedIncidentGroup`. Nous avons aussi eu des informations complémentaires sur l'organisation de la *London Fire Brigade* concernant les casernes, ce qui nous amené à revoir nos variables explicatives tardivement.

Nous avons choisi de présenter ce notebook car il montre un travail de réduction des variables par Analyse en Correspondance Multiple (ACM). Si nous avons abandonné par la suite cette piste pour réduire le nombre de variables explicatives, nous souhaitions en montrer le principe et les résultats.

# <font color='red'>1) Préparation de l'environement de travail </font>

## <font color='blue'>Installation des modules </font>

In [ ]:
#!pip install matplotlib
#!pip install Seaborn
#!pip install openpyxl
#!pip install scipy
#!pip install geopandas
#!pip install scikit-learn
#!pip install statsmodels
#!pip install folium
#!pip install plotly
#!pip install --upgrade seaborn
#!pip install jupyter
#!pip install nbformat
!pip install fanalysis
!pip install torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 1.6 MB/s eta 0:00:00


## <font color='blue'>Importation des bibliothèques </font>

In [ ]:
import pandas as pd  #Pour les dataframe
import numpy as np #Pour le calcul numérique
import datetime as dt # Pour le calcul sur les dates

# Normalisation pour preprocessing
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

# Libraries divers
from copy import deepcopy  # gestion des copies
from pyproj import Proj # conversion entre les coordonnées British national grid et latitude/longitude
from scipy import stats # notamment pour boxplot

# Pour la séparation du jeu de données
from sklearn.model_selection import train_test_split

# Pour réduction de dimension
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from fanalysis.mca import MCA
%matplotlib inline

# Enregistrement du model
import torch


## <font color='blue'>Liaison avec le drive (pour travailler sur GoogleColab) </font>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# <font color='red'>2) Récupération des données (cf Exploration de données)</font>

Code pour travailler sur GoogleColab

In [ ]:
dfim = pd.read_csv('/content/gdrive/My Drive/1_Rendu/InitialDatasets/LFB incident et mobilisation data.csv', low_memory=False)

# modification du type pour les colonnes date
dfim.DateOfCall=pd.to_datetime(dfim.DateOfCall)
dfim.DateAndTimeMobilised=pd.to_datetime(dfim.DateAndTimeMobilised)
dfim.DateAndTimeMobile=pd.to_datetime(dfim.DateAndTimeMobile)
dfim.DateAndTimeArrived=pd.to_datetime(dfim.DateAndTimeArrived)

# df_2023=dfim[dfim.CalYear==2023].copy(deep=True)

Code pour travail en local

In [ ]:
dfim = pd.read_csv('../Data/Datapreprocessing/LFB incident et mobilisation data.csv',low_memory=False)

# modification du type pour les colonnes date
dfim.DateOfCall=pd.to_datetime(dfim.DateOfCall)
dfim.DateAndTimeMobilised=pd.to_datetime(dfim.DateAndTimeMobilised)
dfim.DateAndTimeMobile=pd.to_datetime(dfim.DateAndTimeMobile)
dfim.DateAndTimeArrived=pd.to_datetime(dfim.DateAndTimeArrived)

# df_2023=dfim[dfim.CalYear==2023]

# <font color='red'> 3) Jeux de données pour la modélisation (avant split) </font>

## <font color='blue'>3.a) Transformation Box-Cox </font>

On a choisi de calculer le lambda de la transformation pour chaque variable sur l'ensemble des données (`dfim`) et non sur la seule année 2023 (`df_2023` précédemment étudié).

On obtient :
- pour `TurnoutTimeSeconds` un lambda de 0,4220
- pour `TravelTimeSeconds` un lambda de 0,5073
- pour `TotalResponseTime` un lambda de 0,4589

In [ ]:
# Liste des variables à transformer
variables = ['TurnoutTimeSeconds', 'TravelTimeSeconds', 'TotalResponseTime']

# Appliquer la transformation de Box-Cox avec lambda_optimal à toutes les variables
for variable in variables:
    dfim[f'boxcox_{variable}'], lambda_optimal = stats.boxcox(dfim[variable])
    print(f"Lambda optimal pour la variable : {variable} est {lambda_optimal}")


Lambda optimal pour la variable : TurnoutTimeSeconds est 0.42199745688140766
Lambda optimal pour la variable : TravelTimeSeconds est 0.5073020319480925
Lambda optimal pour la variable : TotalResponseTime est 0.4588683833483651


In [ ]:
# statistiques descriptives des variables de temps et de leur transformations Box-Cox
# nb :.apply(lambda s: s.apply('{0:.2f}'.format)) permet d'afficher les valeurs arrondis à la décimale plutôt que d'avoir une notation scientifique
dfim[['TurnoutTimeSeconds', 'boxcox_TurnoutTimeSeconds','TravelTimeSeconds', 'boxcox_TravelTimeSeconds', 'TotalResponseTime', 'boxcox_TotalResponseTime']].describe().apply(lambda s: s.apply('{0:.2f}'.format))

,TurnoutTimeSeconds,boxcox_TurnoutTimeSeconds,TravelTimeSeconds,boxcox_TravelTimeSeconds,TotalResponseTime,boxcox_TotalResponseTime
count,1037713.00,1037713.00,1037713.00,1037713.00,1037713.00,1037713.00
mean,72.57,11.65,245.51,29.07,318.07,27.86
std,38.32,3.02,129.94,8.49,131.89,5.68
min,1.00,0.00,1.00,0.00,2.00,0.82
25%,51.00,10.08,161.00,23.99,234.00,24.46
50%,69.00,11.78,225.00,28.79,298.00,27.58
75%,88.00,13.31,304.00,33.86,377.00,30.97
max,1192.00,44.71,1181.00,69.36,1200.00,54.22


## <font color='blue'>3.b) Résumé analyse (notebook 2) </font>

Suite aux analyses effectuées sur les variables explicatives connues *a priori*, nous incluerons dans le modèle
- l'heure de l'incident, `HourOfCall`
- l'arrondissement, `IncGeo_BoroughName` (le quartier, `IncGeo_WardName`,  pourra être étudié dans un second temps)
- la caserne de départ, `DeployedFromStation_Name`
- le type d'incident détaillé, `DetailedIncidentGroup` (après regroupement des catégories *False alarm Malicious* / *Good intent* dans une catégorie *Other*)
- le type de propriété impacté dans l'incident, `PropertyCategory` (le type détaillé `HighPropertyType` pourra être étudié dans un second temps)
- la distance
</br></br>

*Nota Bene* :
</br>
En plus des variables connues *a posteriori* (exactitude de l'adresse, délai, nombre d'appels et nombre de camions appelés sur l'incident), les variables suivantes de `df_2023` n'ont pas fait l'objet d'une analyse
- les dates permettant le calcul du temps de trajet et de réponse : `DateAndTimeMobilised`, `DateAndTimeMobile` et `DateAndTimeArrived`
- les variables ayant permis le calcul de la distance : `Latitude`, `Longitude`, `Easting_rounded`, `Northing_rounded`, `Lat_station` et `Long_station`
- la date exacte de l'incident (`DateOfCall`). Nous avons étudié des variables dérivées comme le jour de la semaine ou le mois et nous avons supposé que l'année n'avait pas d'impact sur le temps de réponse (hypothèse soutenue par nos travaux dans la section 3)
- les variables `IncidentStationGround`, `PumpCount`, `ResourceMobilisationId` et `Resource_Code`. Nous avons un doute sur leur définition et sommes en attente d'un retour à ce sujet (note du 15/11/2024). Si notre compréhension est correct, nous ne pensons pas qu'elles puissent avoir un impact sur le temps de réponse.
- la variable `PropertyType` dont `HighPropertyType` est un regroupement



## <font color='blue'>3.c) Création jeux de données (avant split) </font>

Nous créons 2 jeux de données pour la modélisation.

Dans le premier nous incluons
- les variables explicatives sélectionnées dont `IncGeo_WardName` et `HighPropertyType` mais après la modification de `DetailedIncidentGroup`
- les trois variables de temps sur l'échelle originale et la transformation Box-Cox du temps de réponse total (variable à prédire)
- certaines variables qui nous semble interessant de conserver pour des travaux futurs comme la latitude et la longitude par exemple

Dans le second, nous sélectionnons uniquement la variable à prédire (soit la transformation Box-Cox du temps de réponse total) et les variables explicatives retenues pour la première étape de modélisation (sélection du type de modèles); les variables `IncGeo_WardName` et `HighPropertyType` ne sont donc pas dans ce jeu de données.

### Modification de DetailedIncidentGroup
- on conserve le type *AFA* (alarme incendie automatique)
- on regroupe *False alarm - Good intent* et  *False alarm - Malicious* avec *No action (not false alarm)* en renommant *Other* car dans les 3 cas, il n'y a pas eu d'action à mener
- on regroupe les deux types de *Medical Incident* en une catégorie
- on renomme *Late Call* par *Other Fire* (nom plus "parlant")

Nota Bene : *RTC* signifie *Road Traffic Collision*

In [ ]:
dfim.loc[ (dfim.DetailedIncidentGroup=='False alarm - Good intent') | (dfim.DetailedIncidentGroup=='False alarm - Malicious') | (dfim.DetailedIncidentGroup=='No action (not false alarm)'),'DetailedIncidentGroup']='Other'

dfim.loc[ (dfim.DetailedIncidentGroup=='Medical Incident - Co-responder'),'DetailedIncidentGroup']='Medical Incident'
dfim.loc[ (dfim.DetailedIncidentGroup=='Late Call'),'DetailedIncidentGroup']='Other Fire'


# on vérifie la prise en compte des modifications
dfim.DetailedIncidentGroup.value_counts(normalize=True)

,proportion
DetailedIncidentGroup,
AFA,0.401646
Other,0.144326
Primary Fire,0.087047
Secondary Fire,0.086429
Effecting entry/exit,0.080042
Flooding,0.067846
RTC,0.037658
Lift Release,0.016336
Assist other agencies,0.016259


### Catégorisation du temps de réponse

Au cours de notre modélisation, nous souhaitons pouvoir tester des algorithmes de classification. Nous partons de l'hypothèse qu'être capable de prédire un intervalle de temps de réponse pourrait être suffisant. On constate que 98% des temps de réponse sont entre 1 minutes 19 secondes et 13 minutes. Nous avons choisi de créer des catégories de temps correspondant à un intervalle allant de 30 secondes à 6 minutes 30 pour prendre en compte la distribution et avoir au minimum 5% des données dans une catégorie (= un intervalle de temps).

In [ ]:
dfim['TotalResponseTime'].quantile([0.01, 0.025, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.975, 0.99])

,TotalResponseTime
0.010,79.0
0.025,125.0
0.050,152.0
0.100,182.0
0.200,219.0
0.300,247.0
0.400,273.0
0.500,298.0
0.600,325.0
0.700,357.0


Chaque categorie de `ResponseTimeCategory` représente de 4,8 à 11,8% des données. Nous avons choisi les intervalles de temps suivant :
<br> <br>

<table>
<tr>
<th>Categorie</th>
<th>Intervalle</th>
</tr>
<tr>
<td>0</td>
<td> <= 2 minutes 30 secondes </td>
</tr>
<tr>
<td>1</td>
<td>] 2,5 min ; 3 min]</td>
</tr>
<tr>
<td>2</td>
<td>] 3 min; 3,5 min] </td>
</tr>
<tr>
<td>3</td>
<td>] 3,5 min; 4 min ]</td>
</tr>
<tr>
<td>4</td>
<td>] 4 min; 4,5 min]</td>
</tr>
<tr>
<td>5</td>
<td> ] 4,5 min; 5 min ]</td>
</tr>

<tr>
<td>6</td>
<td> ] 5 min; 5,5 min ]</td>
</tr>

<tr>
<td>7</td>
<td> ] 5,5 min; 6 min ]</td>
</tr>

<tr>
<td>8</td>
<td> ] 6 min; 6,5 min ]</td>
</tr>

<tr>
<td>9</td>
<td> ] 6,5 min; 7,5 min ]</td>
</tr>

<tr>
<td>10</td>
<td> ] 7,5 min; 9 min ]</td>
</tr>

<tr>
<td>11</td>
<td> > 9 minutes </td>
</tr>

</table>

In [ ]:
dfim['ResponseTimeCategory']=0
inf=[150, 180, 210, 240, 270, 300, 330, 360, 390, 450, 540]
sup=[180, 210, 240, 270, 300, 330, 360, 390, 450, 540, 630]

j=0
for i in range(0,11):
  j=j+1
  dfim.loc[(dfim.TotalResponseTime>inf[i]) & (dfim.TotalResponseTime<=sup[i]), 'ResponseTimeCategory']=j

dfim.loc[(dfim.TotalResponseTime>=630), 'ResponseTimeCategory']=11


In [ ]:
dfim.ResponseTimeCategory.value_counts(normalize=True)

,proportion
ResponseTimeCategory,
5,0.118373
4,0.116337
6,0.107992
3,0.101335
9,0.095471
7,0.091508
2,0.076452
8,0.071000
10,0.069274


### Création d'un premier jeu de données "clean"
On supprime les variables
- connues *a posteriori*
- dont la définition n'est pas assez précise
- de dates permettant le calcul du temps de trajet et de réponse

Ce jeu de données a 1 037 713 lignes et 28 colonnes. On renomme certaines variables.

In [ ]:
col=['IncidentNumber', 'TurnoutTimeSeconds', 'TravelTimeSeconds', 'TotalResponseTime', 'boxcox_TotalResponseTime', 'ResponseTimeCategory',
       'DateOfCall', 'CalYear', 'TimeOfCall', 'HourOfCall', 'DayOfWeek', 'Month',
       'IncidentGroup', 'DetailedIncidentGroup',
       'PropertyCategory', 'HighPropertyType',
       'IncGeo_BoroughCode','IncGeo_BoroughName', 'IncGeo_WardCode', 'IncGeo_WardName',
       'DeployedFromStation_Code', 'DeployedFromStation_Name',
       'distance', 'Latitude', 'Longitude', 'IncGeo_Rounded','Lat_station', 'Long_station',
    ]

df_clean=dfim[col].copy(deep=True)

# on renomme certaines colonnes

df_clean.rename(columns={'TurnoutTimeSeconds': 'TurnoutTime',
                             'TravelTimeSeconds' : 'TravelTime',
                             'boxcox_TotalResponseTime' : 'TotalResponseTime_BC',
                             'IncGeo_BoroughCode' : 'BoroughCode',
                             'IncGeo_BoroughName' : 'BoroughName',
                             'IncGeo_WardCode' : 'WardCode',
                             'IncGeo_WardName' : 'WardName',
                             'DeployedFromStation_Code' : 'Station_Code',
                             'DeployedFromStation_Name' : 'Station_Name',
                              'DetailedIncidentGroup' : 'Incident_Type'
                            }, inplace=True)

In [ ]:
df_clean.shape

(1037713, 28)

Enregistrement sur Google Colab

In [ ]:
df_clean.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/CleanDataset.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
df_clean.to_csv('../Data/Datapreprocessing/Complete//CleanDataset.csv', index=False , encoding='utf-8')

Lecture sur Google Colab

In [ ]:
df_clean = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/CleanDataset.csv', low_memory=False)

# apres le chargement, il faut modifier le type des dates
df_clean.DateOfCall=pd.to_datetime(df_clean.DateOfCall)

Lecture en local

In [ ]:
df_clean = pd.read_csv('../Data/Datapreprocessing/Complete/CleanDataset.csv',low_memory=False)

# apres le chargement, il faut modifier le type des dates
df_clean.DateOfCall=pd.to_datetime(df_clean.DateOfCall)

### Création d'un second jeu de données "minimal" pour la modélisation
- On supprime les variables qui ne seront pas utilisées dans la première étape de modélisation (choix du type de modèle) sauf `CalYear` car on va utiliser cette variable pour créer les datasets d'entraînement / validation / test et `IncidentNumber` pour pouvoir faire le lien avec le dataset complet.
- on binarise les variables catégorielles nominales
- on crée 2 variables binaires pour les heures : `H26` qui vaut 1 entre 2 et 6 heures et 0 sinon et `H1117` qui vaut 1 entre 11 et 17 heures et 0 sinon.


*Nota Bene* : à ce stade, on conserve les 2 variables explicatives possibles `TotalResponseTime_BC` et `ResponseTimeCategory`

Ce jeu de données a 1 037 713 lignes et 177 colonnes (après la suppression de `HourOfCall`)

In [ ]:
# selection des colonnes
col=[ 'IncidentNumber', 'TotalResponseTime_BC', 'ResponseTimeCategory', 'CalYear',
       'HourOfCall',
       'Incident_Type', 'PropertyCategory',
       'BoroughCode',
       'Station_Code', 'distance']

df_model=df_clean[col].copy(deep=True)


# Création d'une variable qualitative de l'heure d'appel
df_model['HCat']='H'
df_model.loc[(df_model.HourOfCall>=2) & (df_model.HourOfCall<=6),'HCat']='H26'
df_model.loc[(df_model.HourOfCall>=11) & (df_model.HourOfCall<=17),'HCat']='H1117'

# binarisation
#df_model=pd.get_dummies(data=df_model, columns=['Incident_Type', 'PropertyCategory', 'BoroughCode','Station_Code'], prefix=['IncTyp', 'PropCat','Borough', 'Station'], prefix_sep='_')


In [ ]:
df_model.shape

(1037713, 11)

In [ ]:
# on supprime la colonne avec les heures
df_model.drop(columns=['HourOfCall'], inplace=True)

df_model.shape

(1037713, 10)

Enregistrement sur Google Colab

In [ ]:
df_model.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/ModelingDataset_nobin.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
df_model.to_csv('../Data/Datapreprocessing/Complete/ModelingDataset_nobin.csv', index=False , encoding='utf-8')

Lecture sur Google Colab

In [ ]:
df_model = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/ModelingDataset_nobin.csv', low_memory=False)



Lecture en local

In [ ]:
df_model = pd.read_csv('../Data/Datapreprocessing/Complete/ModelingDataset_nobin.csv', low_memory=False)



## <font color='blue'>3.d) réduction à l'aide d'une Analyse en Correspondance Multiple </font>

L'Analyse en Correspondance Multiple (ACM) est une méthode statistique qui permet de
* de visualiser des variables qualitatives sur un nombre de dimensions réduits
* réduire la complexité en transformant un ensemble de variables en un ensemble réduit tout en conservant un maximum d'information.

L'ACM est accessible directement en Python à l'aide du package **fanalysis** ( pour plus d'explication voir [ici](https://github.com/OlivierGarciaDev/fanalysis/blob/master/doc/mca_tutorial.ipynb)).


Pour réduire le nombre de variables explicatives de notre jeux de données, nous avons réalisé les étapes suivantes :

1.   Réalisation de l'ACM sur l'ensemble des variables qualitatives de notre dataframe `df_model`
2.   Recherche du nombre d'axes permettant de conserver 95% de l'information.
3.   Construction d'une nouvelle ACM avec le nombre d'axes fixé au nombre trouvé précédement.
4.   Entrainement sur le jeu de données de cette nouvelle ACM
5.   Jointure avec la colonne quantitative `distance` et les colonnes cibles `TotalResponseTime_BC`,`ResponseTimeCategory`

La fonction renvoi aussi le transformateur mca, pour réentrainer des données au besoin.

In [ ]:
df_model.columns

Index(['IncidentNumber', 'TotalResponseTime_BC', 'ResponseTimeCategory',
       'CalYear', 'Incident_Type', 'PropertyCategory', 'BoroughCode',
       'Station_Code', 'distance', 'HCat'],
      dtype='object')

In [ ]:
df_model.shape

(1037713, 10)

In [ ]:
def entrainement_model(X):
    '''
    Construction d'une MCA pour garder 95% de l'information du dataFrame

    Arg : X -> dataFrame
    '''
    # Création de l'objet MCA et ajustement du modèle
    mca = MCA()

    # Création d'une matrice (tableau de tableau) contenant la description de chaque individus
    X = X.values

    # Entrainement de la MCA sur X
    mca.fit(X)

    # Récupération de l'indice pour 95% des informations
    arg_max =np.argmax(mca.eig_[2,:]>=95)

    del mca

    # Création de la nouvelle ACM
    mca= MCA(n_components=arg_max+1,stats=False)

    # Entrainement du modèle
    mca.fit(X)

    return mca, arg_max

def transfo_temp(mca,X,arg_max):
    '''
    Transformation grace à la MCA du dataFrame par jeu de 50 000 lignes

    Args :
        mca -> transformateur de MCA
        X dataFrame
    '''
    df_mca = pd.DataFrame()
    i=0

    for i in range(0,(X.shape[0]//50000)):
        # Application de de la MCA à df
        Y = mca.transform(X.iloc[i*50000:(i+1)*50000,:].values)

        # Transformation en dataFrame
        Y = pd.DataFrame(Y).iloc[:,:arg_max]

        # Concatenation
        df_mca = pd.concat([df_mca,Y],axis=0)


    # Application de de la MCA à df
    Y = mca.transform(X.iloc[(i+1)*50000:,:].values)

    # Transformation en dataFrame
    Y = pd.DataFrame(Y).iloc[:,:arg_max]

    # Concatenation
    df_mca = pd.concat([df_mca,Y],axis=0)

    return df_mca

def ajout_col(X_mca,X):
    '''
    Ajout des variables cibles et distance

    Args : X_mca : dataframe réduit
            X : dataframe original (doit contenir 'distance','TotalResponseTime_BC','ResponseTimeCategory' )
    '''
    # Ajouter la colonne 'distStd'
    X_mca['distance'] = X['distance']

    # Ajouter la colonne 'TotalResponseTime_BC'
    X_mca.insert(0, 'TotalResponseTime_BC', X['TotalResponseTime_BC'])

    # Ajouter la colonne 'ResponseTimeCategory'
    X_mca.insert(1, 'ResponseTimeCategory', X['ResponseTimeCategory'])

    return X_mca

In [ ]:
def transfo_mca(col_desc_quali,df):
    '''
    Appliquer une réduction de dimension par ACM en conservant 95% des informations

    Args:
        col_desc_quali -> list : nom des variables qualitatives
        df_train, df_validation, df_test -> dataFrame d'entrainement, de validation et de test.
        Ces trois dataFrame doivent contenir les variables 'TotalResponseTime_BC', 'ResponseTimeCategory','distStd' et les variables contenues dans col_desc_quali
    '''
    # Entrainement du modèle
    mca, arg_max = entrainement_model(df.loc[:,col_desc_quali])

    # Transformation de X_train
    df_mca = transfo_temp(mca,df.loc[:,col_desc_quali],arg_max)
    df_mca = ajout_col(df_mca,df)

    return mca,df_mca

In [ ]:
col_desc_quali = ['Incident_Type', 'PropertyCategory', 'BoroughCode', 'Station_Code','HCat']

mca,df_mca=transfo_mca(col_desc_quali,df_model.iloc[:100000,:])

Enregistrement sur googlecolab

In [ ]:
df_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/ModelingDataset_mca.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
df_mca.to_csv('../Data/Datapreprocessing/Complete/ModelingDataset_mca.csv', index=False , encoding='utf-8')

Lecture sur googlecolab

In [ ]:
df_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/ModelingDataset_mca.csv', low_memory=False)

Lecture en local

In [ ]:
df_mca = pd.read_csv('../Data/Datapreprocessing/Complete/ModelingDataset_nobin.csv', low_memory=False)

# <font color='red'> 4) Split du jeu de données
 </font>







## <font color='blue'>4.a) Split du dataset "complet" </font>

#### Methode pour le split

In [ ]:
def split_dataframe(df, train_ratio, validation_ratio, test_ratio, random_state=None):
    """
    Divise un DataFrame en trois parties : train, validation, et test.

    Parameters:
    - df (pd.DataFrame): Le DataFrame à diviser.
    - train_ratio (float): Proportion des données pour l'ensemble d'entraînement.
    - validation_ratio (float): Proportion des données pour l'ensemble de validation.
    - test_ratio (float): Proportion des données pour l'ensemble de test.
    - random_state (int, optional): Pour reproduire la même division aléatoire.

    Returns:
    - train_df (pd.DataFrame): Jeu d'entraînement.
    - validation_df (pd.DataFrame): Jeu de validation.
    - test_df (pd.DataFrame): Jeu de test.
    """
    total_ratio = train_ratio + validation_ratio + test_ratio
    assert abs(total_ratio - 1.0) < 1e-6, f"Les proportions doivent totaliser 1. Actuellement : {total_ratio}"

    # Mélanger les données
    df = df.sample(frac=1, random_state=random_state).reset_index(drop=True)

    # Étape 1 : Diviser en train + temp (validation + test)
    train_df, temp_df = train_test_split(df, test_size=(1 - train_ratio), random_state=random_state)

    # Étape 2 : Diviser temp en validation et test
    validation_df, test_df = train_test_split(temp_df,
                                              test_size=test_ratio / (test_ratio + validation_ratio),
                                              random_state=random_state)

    return train_df, validation_df, test_df

#### Split du jeu données

Dans un premier temps, on divise le jeu de données `df_model` en 3 jeux de données pour l'entraînement, la validation et le test de modèles. grâce à la fonction `split_dataframe`. Nous utilisons `random_state=2024` pour pouvoir reproduire ce split.

*Nota Bene* : nous avons vérifié et la répartition des incidents par `CalYear` est similaire dans les 3 jeux de données.



In [ ]:
train_df, validation_df, test_df = split_dataframe(df_model, 0.7, 0.15, 0.15, 2024)

# regroupement de train et validation pour la dernière étape de modélisation
train2_df = pd.concat([train_df, validation_df])

## df_model.CalYear.value_counts(normalize=True)
## train_df.CalYear.value_counts(normalize=True)
## validation_df.CalYear.value_counts(normalize=True)
## test_df.CalYear.value_counts(normalize=True)

#### Standardisation de la distance

Une fois le jeu de données divisé, nous standardisons la variable quantitative `distance`. Comme nous avons pu le constater dans le notebook précédent, la répartition de la distance est asymétrique avec une médiane à 1,4 km et un maximum à 40,3 km. Nous avons donc choisi la méthode de standardisation `RobustScaler` qui utilise la médiane et l'interquartile.

Ci-dessous, nous avons appliqué les 3 standardisations à l'ensemble des données de `df_model` pour comparer leurs statistiques descriptives. Cela confirme que la méthode `RobustScaler` est la plus adaptée.

In [ ]:
# statistique descriptive sur la distance
tempo = df_model[['distance']].copy(deep=True)
tempo['dist_rob']=pd.DataFrame(RobustScaler().fit_transform(tempo[['distance']]))
tempo['dist_norm']=pd.DataFrame(StandardScaler().fit_transform(tempo[['distance']]))
tempo['dist_min']=pd.DataFrame(MinMaxScaler().fit_transform(tempo[['distance']]))



display(tempo[['distance', 'dist_norm', 'dist_rob', 'dist_min']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
#del tempo

,distance,dist_norm,dist_rob,dist_min
count,1037713.00,1037713.00,1037713.00,1037713.00
mean,1754.00,0.00,0.26,0.04
std,1766.98,1.00,1.41,0.04
min,0.20,-0.99,-1.14,0.00
25%,874.81,-0.50,-0.44,0.02
50%,1429.71,-0.18,0.00,0.04
75%,2123.69,0.21,0.56,0.05
max,40325.33,21.83,31.14,1.00


Nous effectuons 2 standardisations en prenant en compte les données du jeu de données train (`train_df`) et celles des 2 jeux de données regroupées `train_df` et `validation_df` (soit `train2_df`). Chaque standardisation sera utilisée à une étape différente de la modélisation.

In [ ]:
# premier cas : on utiliser la médiane et IQR de train_df
scaler = RobustScaler()
train_df['distStd'] = scaler.fit_transform(train_df[['distance']]) # fit et transform sur train_df
validation_df['distStd'] = scaler.transform(validation_df[['distance']]) # uniquement le transform sur validation_df


# second cas : on utilise la médiane et IQR du regroupement de train_df + validation_df
scaler = RobustScaler()
train2_df['distStd'] = scaler.fit_transform(train2_df[['distance']]) # fit et transform sur train2_df
test_df['distStd'] = scaler.transform(test_df[['distance']]) # uniquement le transfomr sur test_df


In [ ]:
# display(train_df[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(validation_df[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

# display(train2_df[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(test_df[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

In [ ]:
# on conserve uniquement la distance standardisée (on pourra retrouver la distance "originale" dans le dataset df_model et df_clean)
# on supprime la colonne CalYear, on conserve IncidentNumber pour pouvoir retrouver les liens avec df_model et df_clean
train_df.drop(columns=['distance', 'CalYear'], inplace=True)
validation_df.drop(columns=['distance', 'CalYear'], inplace=True)
test_df.drop(columns=['distance', 'CalYear'], inplace=True)
train2_df.drop(columns=['distance', 'CalYear'], inplace=True)

#### Enregistrement des différents jeux de données

Enregistrement sur Google Colab

In [ ]:
train_df.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset_nobin.csv', index=False , encoding='utf-8')
validation_df.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset_nobin.csv', index=False , encoding='utf-8')
train2_df.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Step3_Dataset_nobin.csv', index=False , encoding='utf-8')
test_df.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset_nobin.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
train_df.to_csv('../Data/Datapreprocessing/Complete/Train_Dataset_nobin.csv', index=False , encoding='utf-8')
validation_df.to_csv('../Data/Datapreprocessing/Complete/Validation_Dataset_nobin.csv', index=False , encoding='utf-8')
train2_df.to_csv('../Data/Datapreprocessing/Complete/Complete/Train_Step3_Dataset_nobin.csv', index=False , encoding='utf-8')
test_df.to_csv('../Data/Datapreprocessing/Complete/Complete/Test_Dataset_nobin.csv', index=False , encoding='utf-8')

Lecture sur Google Colab

In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset_nobin.csv', low_memory=False)
validation_df = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset_nobin.csv', low_memory=False)
train2_df = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Step3_Dataset_nobin.csv', low_memory=False)
test_df = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset_nobin.csv', low_memory=False)



Lecture en local

In [ ]:
train_df = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Dataset_nobin.csv', low_memory=False)
validation_df = pd.read_csv('../Data/Datapreprocessing/Complete/Validation_Dataset_nobin.csv', low_memory=False)
train2_df = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Step3_Dataset_nobin.csv', low_memory=False)
test_df = pd.read_csv('../Data/Datapreprocessing/Complete/Test_Dataset_nobin.csv', low_memory=False)

## <font color='blue'>4.b) Création de datasets "réduits" </font>

Pour rappel, une fois la binarisation des variables qualitatives effectuée, notre jeu de données avant split contient 1 037 713 lignes et 177 colonnes. Nous nous attendons à des difficultés lors de l'entraînement des modèles (temps d'exécution trop long, problème de convergence pour l'algorithme), surtout pour les plus complexes. Nous créons donc des jeux de données réduits. Pour effectuer la sélection, nous prenons en compte l'année de l'incident. En effet, nous préférons conserver en priorité les données les plus récentes tout en prenant en compte les plus anciennes.

#### Methode pour le split avec réduction

In [ ]:
def split_final(df, ratio_year, ratio_separation, nb_ligne_2024=0, random_state=None):
    """
    Divise un DataFrame par année selon les ratios fournis et retourne 3 DataFrames globaux associé au 3 Series y.

    Parameters:
    - df (pd.DataFrame): Le DataFrame à diviser. Doit contenir la variable CalYear avec des valeurs comprises entre 2014 et 2024
    - ratio_year (list): liste des ratios de valeurs souhaitées dans chaque année
    - ratio_separation (list): liste des ratio train/validation/test
    - random_state (int, optional): Pour reproduire les mêmes divisions aléatoires.

    Returns:
    - train_dfs (pd.DataFrame): Jeu d'entraînement.
    - val_dfs (pd.DataFrame): Jeu de validation.
    - test_dfs (pd.DataFrame): Jeu de test.
    - train_ys (pd.Serie): Résultat du jeu d'entraînement.
    - val_ys (pd.Serie): Résultat du jeu de validation.
    - test_ys (pd.Serie): Résultat du jeu de test.
    """


    # Vérification des ratios
    total_ratio_separation = sum(ratio_separation)
    assert abs(total_ratio_separation - 1.0) < 1e-6, f"Les proportions de séparation doivent totaliser 1. Actuellement : {total_ratio_separation}"
    total_ratio_repartition = sum(ratio_year)
    assert abs(total_ratio_repartition - 1.0) < 1e-6, f"Les proportions de séparation doivent totaliser 1. Actuellement : {total_ratio_repartition}"

    # Filtrer les données pour l'année
    filtered_df = df[df['CalYear'] == 2024]

    ratio_0 = ratio_year[10]
    if (int(nb_ligne_2024*ratio_0)>filtered_df.shape[0]) | (nb_ligne_2024==0):
      nb_ligne = filtered_df.shape[0]
    else:
      nb_ligne = int(nb_ligne_2024*ratio_0)

    # Mélanger les données
    filtered_df = filtered_df.sample(frac=1, random_state=random_state).reset_index(drop=True)
    # on selectionne nb_ligne parmi les lignes de filtered_df
    filtered_d = filtered_df[:int(nb_ligne)]

    # Diviser les données en 3 (train, validation et test)
    train_Xs, val_Xs, test_Xs, = split_dataframe(
        filtered_d,
        train_ratio=ratio_separation[0],
        validation_ratio=ratio_separation[1],
        test_ratio=ratio_separation[2],
        random_state=random_state,
    )

    # liste des années en décroissant
    list_year = [2023 , 2022 , 2021 , 2020 , 2019 , 2018 , 2017 , 2016 , 2015 , 2014]

    for indice, year in enumerate(list_year):
        # Récupérer le ratio pour l'année
        ratio = ratio_year[9-indice]

        # Filtrer les données pour l'année
        filtered_df = df[df["CalYear"] == year]

        # Mélanger les données
        filtered_df = filtered_df.sample(frac=1, random_state=random_state).reset_index(drop=True)

        if int(nb_ligne*ratio/filtered_df.shape[0]) > ratio_0:
            print('trop petit ',list_year[indice])
            break
        else :
            # Récupération du nombre de lignes nécessaires
            filtered_d = filtered_df[:int(nb_ligne*ratio/ratio_0)]

        # Diviser les données
        train_X, val_X, test_X = split_dataframe(
            filtered_d,
            train_ratio=ratio_separation[0],
            validation_ratio=ratio_separation[1],
            test_ratio=ratio_separation[2],
            random_state=random_state,
        )

        # Concaténation des données
        train_Xs = pd.concat([train_Xs , train_X] , axis = 0)
        val_Xs = pd.concat([val_Xs , val_X] , axis = 0)
        test_Xs = pd.concat([test_Xs , test_X] , axis = 0)

    return train_Xs, val_Xs, test_Xs


#### Split du jeu de données

On choisit de faire une première réduction de sorte à faire une sélection sur les années 2014 à 2023 et à avoir la totalité des lignes 2024 (les plus récentes), soit les 92087 incidents.

In [ ]:
# premiere reduction avec la totalité des données sur 2024 (92 087 lignes)
# au total 306 953 lignes (92 087/0.3)
train_reduit1, val_reduit1, test_reduit1 = split_final(
    df_model,
    ratio_separation = [0.7 , 0.15 , 0.15],
    ratio_year = [0.02 , 0.02 , 0.02 , 0.02, 0.02 , 0.05 , 0.05,  0.1 , 0.1 , 0.3, 0.3 ],
    nb_ligne_2024=df_model.shape[0],
    random_state = 2024)

In [ ]:
print('Nb d\'incidents en 2024 :', df_model[df_model.CalYear==2024].shape[0])
print('Nb d\'incidents sur 10 ans avec 30% en 2024 :', np.round(df_model[df_model.CalYear==2024].shape[0]/0.3,0))
print('Nb d\'incidents sélectionnés (vérification) : ', train_reduit1.shape[0] +  test_reduit1.shape[0] + val_reduit1.shape[0])
print('dont 2024 : ', train_reduit1[train_reduit1.CalYear==2024].shape[0] +  test_reduit1[test_reduit1.CalYear==2024].shape[0] + val_reduit1[val_reduit1.CalYear==2024].shape[0])

Nb d'incidents en 2024 : 92087
Nb d'incidents sur 10 ans avec 30% en 2024 : 306957.0
Nb d'incidents sélectionnés (vérification) :  306953
dont 2024 :  92087


In [ ]:
print('Répartition des incidents par année (vérification sur train) :\n')
display(train_reduit1.CalYear.value_counts(normalize=True))

Répartition des incidents par année (vérification sur train) :



,proportion
CalYear,
2024,0.300008
2023,0.300008
2022,0.100000
2021,0.100000
2020,0.049995
2019,0.049995
2018,0.019999
2017,0.019999
2016,0.019999


Nous avons fait une seconde réduction de la taille du jeu de données de sorte à limiter la taille à 200 000 incidents. Pour maximiser la prise en compte des incidents 2024, nous avons modifié les pourcentages.

In [ ]:
train_reduit2, val_reduit2, test_reduit2 = split_final(
    df_model,
    ratio_separation = [0.7 , 0.15 , 0.15],
    ratio_year = [0.02 , 0.02 , 0.02 , 0.02, 0.02 , 0.05 , 0.05,  0.075 , 0.075 , 0.25, 0.4],
    nb_ligne_2024=200000,
    random_state = 2024)

In [ ]:
print('Nb d\'incidents en 2024 :', df_model[df_model.CalYear==2024].shape[0])
print('Nb d\'incidents sélectionnés (vérification) : ', train_reduit2.shape[0] +  test_reduit2.shape[0] + val_reduit2.shape[0])
print('dont 2024 : ', train_reduit2[train_reduit2.CalYear==2024].shape[0] +  test_reduit2[test_reduit2.CalYear==2024].shape[0] + val_reduit2[val_reduit2.CalYear==2024].shape[0])

Nb d'incidents en 2024 : 92087
Nb d'incidents sélectionnés (vérification) :  200000
dont 2024 :  80000


In [ ]:
print('Répartition des incidents par année (vérification sur train) :\n')
display(train_reduit2.CalYear.value_counts(normalize=True))

Répartition des incidents par année (vérification sur train) :



,proportion
CalYear,
2024,0.400024
2023,0.250013
2022,0.074999
2021,0.074999
2020,0.049997
2019,0.049997
2018,0.019994
2017,0.019994
2016,0.019994


#### Standardisation de la distance

In [ ]:
# regroupement de train et validation pour la dernière étape de modélisation
train2_reduit1 = pd.concat([train_reduit1, val_reduit1])
train2_reduit2 = pd.concat([train_reduit2, val_reduit2])

In [ ]:
# premier cas : on utiliser la médiane et IQR de train_df
scaler = RobustScaler()
train_reduit1['distStd'] = scaler.fit_transform(train_reduit1[['distance']]) # fit et transform sur train_df
val_reduit1['distStd'] = scaler.transform(val_reduit1[['distance']]) # uniquement le transfomr sur validation_df


# second cas : on utilise la médiane et IQR du regroupement de train_df + validation_df
scaler = RobustScaler()
train2_reduit1['distStd'] = scaler.fit_transform(train2_reduit1[['distance']]) # fit et transform sur train2_df
test_reduit1['distStd'] = scaler.transform(test_reduit1[['distance']]) # uniquement le transfomr sur test_df

In [ ]:
# premier cas : on utiliser la médiane et IQR de train_df
scaler = RobustScaler()
train_reduit2['distStd'] = scaler.fit_transform(train_reduit2[['distance']]) # fit et transform sur train_df
val_reduit2['distStd'] = scaler.transform(val_reduit2[['distance']]) # uniquement le transfomr sur validation_df


# second cas : on utilise la médiane et IQR du regroupement de train_df + validation_df
scaler = RobustScaler()
train2_reduit2['distStd'] = scaler.fit_transform(train2_reduit2[['distance']]) # fit et transform sur train2_df
test_reduit2['distStd'] = scaler.transform(test_reduit2[['distance']]) # uniquement le transfomr sur test_df

In [ ]:
# display(train_reduit1[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(val_reduit1[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

# display(train2_reduit1[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(test_reduit1[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

# display(train_reduit2[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(val_reduit2[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

# display(train2_reduit2[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))
# display(test_reduit2[['distance', 'distStd']].describe().apply(lambda s: s.apply('{0:.2f}'.format)))

In [ ]:
# on conserve uniquement la distance standardisée (on pourra retrouver la distance "originale" dans le dataset df_model et df_clean)
# on supprime la colonne CalYear, on conserve IncidentNumber pour pouvoir retrouver les liens avec df_model et df_clean
train_reduit1.drop(columns=['distance', 'CalYear'], inplace=True)
val_reduit1.drop(columns=['distance', 'CalYear'], inplace=True)
test_reduit1.drop(columns=['distance', 'CalYear'], inplace=True)
train2_reduit1.drop(columns=['distance', 'CalYear'], inplace=True)

train_reduit2.drop(columns=['distance', 'CalYear'], inplace=True)
val_reduit2.drop(columns=['distance', 'CalYear'], inplace=True)
test_reduit2.drop(columns=['distance', 'CalYear'], inplace=True)
train2_reduit2.drop(columns=['distance', 'CalYear'], inplace=True)

#### Enregistrement des différents jeux de données

Enregistrement sur Google Colab

In [ ]:
train_reduit1.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Train_Dataset_nobin.csv', index=False , encoding='utf-8')
val_reduit1.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Validation_Dataset_nobin.csv', index=False , encoding='utf-8')
train2_reduit1.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Train_Step3_Dataset_nobin.csv', index=False , encoding='utf-8')
test_reduit1.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Test_Dataset_nobin.csv', index=False , encoding='utf-8')

train_reduit2.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Dataset_nobin.csv', index=False , encoding='utf-8')
val_reduit2.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Validation_Dataset_nobin.csv', index=False , encoding='utf-8')
train2_reduit2.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Step3_Dataset_nobin.csv', index=False , encoding='utf-8')
test_reduit2.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Test_Dataset_nobin.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
train_reduit1.to_csv('../Data/Datapreprocessing/Reduit1/Train_Dataset_nobin.csv', index=False , encoding='utf-8')
val_reduit1.to_csv('../Data/Datapreprocessing/Reduit1/Validation_Dataset_nobin.csv', index=False , encoding='utf-8')
train2_reduit1.to_csv('../Data/Datapreprocessing/Reduit1/Train_Step3_Dataset_nobin.csv', index=False , encoding='utf-8')
test_reduit1.to_csv('../Data/Datapreprocessing/Reduit1/Test_Dataset_nobin.csv', index=False , encoding='utf-8')

train_reduit2.to_csv('../Data/Datapreprocessing/Reduit2/Train_Dataset_nobin.csv', index=False , encoding='utf-8')
val_reduit2.to_csv('../Data/Datapreprocessing/Reduit2/Validation_Dataset_nobin.csv', index=False , encoding='utf-8')
train2_reduit2.to_csv('../Data/Datapreprocessing/Reduit2/Train_Step3_Dataset_nobin.csv', index=False , encoding='utf-8')
test_reduit2.to_csv('../Data/Datapreprocessing/Reduit2/Test_Dataset_nobin.csv', index=False , encoding='utf-8')

Lecture depuis Google Colab

In [ ]:
train_reduit1 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Train_Dataset_nobin.csv', low_memory=False)
val_reduit1 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Validation_Dataset_nobin.csv', low_memory=False)
train2_reduit1 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Train_Step3_Dataset_nobin.csv', low_memory=False)
test_reduit1 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit1/Test_Dataset_nobin.csv', low_memory=False)

train_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Dataset_nobin.csv', low_memory=False)
val_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Validation_Dataset_nobin.csv', low_memory=False)
train2_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Train_Step3_Dataset_nobin.csv', low_memory=False)
test_reduit2 = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Reduit2/Test_Dataset_nobin.csv', low_memory=False)

Lecture depuis un enregistrement en local

In [ ]:
train_reduit1 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Train_Dataset_nobin.csv', low_memory=False)
val_reduit1 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Validation_Dataset_nobin.csv', low_memory=False)
train2_reduit1 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Train_Step3_Dataset_nobin.csv', low_memory=False)
test_reduit1 = pd.read_csv('../Data/Datapreprocessing/Reduit1/Test_Dataset_nobin.csv', low_memory=False)

train_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit2/Train_Dataset_nobin.csv', low_memory=False)
val_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit2/Validation_Dataset_nobin.csv', low_memory=False)
train2_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit2/Train_Step3_Dataset_nobin.csv', low_memory=False)
test_reduit2 = pd.read_csv('../Data/Datapreprocessing/Reduit2/Test_Dataset_nobin.csv', low_memory=False)

## <font color='blue'>4.c) Réduction de dimension avec l'ACM </font>

**Présentation** une Analyse en Correspondance Multiple (ACM) est une méthode statistique qui permet d'explorer et de visualiser des données qualitatives.

**Objectifs :** Réduire la complexité : Elle transforme un tableau en un tableau de donnée moins grans tout en conservant un maximum d'information.

L'ACM est accéssible directement en Python à l'aide du package **fanalysis** (lien pour explication : https://github.com/OlivierGarciaDev/fanalysis/blob/master/doc/mca_tutorial.ipynb).


Pour réduire la dimension de notre DataFrame nous fonctionnerons de la manière suivante :
- Sur le jeu d'entrainement :
    - Création de la MCA sur tout le jeu de données réduit aux variables qualitatives.
    - Recherche du nombre d'axe permettant de conserver 95% de l'information.
    - Construction d'une nouvelle MCA avec le nombre d'axe fixé au nombre trouvé précédement.
    - Entrainement su jeu de données sur la nouvelle MCA
    - Jointure avec la colonne quantitative 'distance'
-Sur les jeux de donnée de validation et de test
    - Entrainement sur la MCA
    - Jointure avec la colonne quantitative 'distance'

La fonction renvoi aussi le transformateur mca, pour réentrainer des données au besoin.

In [ ]:
def entrainement_model(X):
    '''
    Construction d'une MCA pour garder 95% de l'information du dataFrame

    Arg : X -> dataFrame
    '''
    # Création de l'objet MCA et ajustement du modèle
    mca = MCA()

    # Création d'une matrice (tableau de tableau) contenant la description de chaque individus
    X = X.values

    # Entrainement de la MCA sur X
    mca.fit(X)

    # Récupération de l'indice pour 95% des informations
    arg_max =np.argmax(mca.eig_[2,:]>=95)

    del mca

    # Création de la nouvelle ACM
    mca= MCA(n_components=arg_max+1,stats=False)

    # Entrainement du modèle
    mca.fit(X)

    return mca, arg_max

def transfo_temp(mca,X,arg_max):
    '''
    Transformation grace à la MCA du dataFrame par jeu de 50 000 lignes

    Args :
        mca -> transformateur de MCA
        X dataFrame
    '''
    df_mca = pd.DataFrame()
    i=0

    for i in range(0,(X.shape[0]//50000)):
        # Application de de la MCA à df
        Y = mca.transform(X.iloc[i*50000:(i+1)*50000,:].values)

        # Transformation en dataFrame
        Y = pd.DataFrame(Y).iloc[:,:arg_max]

        # Concatenation
        df_mca = pd.concat([df_mca,Y],axis=0)


    # Application de de la MCA à df
    Y = mca.transform(X.iloc[(i+1)*50000:,:].values)

    # Transformation en dataFrame
    Y = pd.DataFrame(Y).iloc[:,:arg_max]

    # Concatenation
    df_mca = pd.concat([df_mca,Y],axis=0)

    return df_mca

def ajout_col(X_mca,X):
    '''
    Ajout des variables cibles et distance

    Args : X_mca : dataframe réduit
            X : dataframe original (doit contenir 'distStd','TotalResponseTime_BC','ResponseTimeCategory' )
    '''
    # Ajouter la colonne 'distStd'
    X_mca['distStd'] = X['distStd']

    # Ajouter la colonne 'TotalResponseTime_BC'
    X_mca.insert(0, 'TotalResponseTime_BC', X['TotalResponseTime_BC'])

    # Ajouter la colonne 'ResponseTimeCategory'
    X_mca.insert(1, 'ResponseTimeCategory', X['ResponseTimeCategory'])

    return X_mca


In [ ]:
def transfo_mca(col_desc_quali,X_train,X_val,X_test):
    '''
    Appliquer une réduction de dimension par ACM en conservant 95% des informations

    Args:
        col_desc_quali -> list : nom des variables qualitatives
        df_train, df_validation, df_test -> dataFrame d'entrainement, de validation et de test.
        Ces trois dataFrame doivent contenir les variables 'TotalResponseTime_BC', 'ResponseTimeCategory','distStd' et les variables contenues dans col_desc_quali
    '''
    # Entrainement du modèle
    mca, arg_max = entrainement_model(X_train.loc[:,col_desc_quali])

    # Transformation de X_train
    X_train_mca = transfo_temp(mca,X_train.loc[:,col_desc_quali],arg_max)
    X_train_mca = ajout_col(X_train_mca,X_train)

    # Transformation de X_validation
    X_val_mca = transfo_temp(mca,X_val.loc[:,col_desc_quali],arg_max)
    X_val_mca = ajout_col(X_val_mca,X_val)

    # Transformation de X_test
    X_test_mca = transfo_temp(mca,X_test.loc[:,col_desc_quali],arg_max)
    X_test_mca = ajout_col(X_test_mca,X_test)

    return mca,X_train_mca,X_val_mca,X_test_mca

##### Pour jeux complets

In [ ]:
# Tourne pendant 15 min

col_desc_quali = ['Incident_Type', 'PropertyCategory', 'BoroughCode', 'Station_Code','HCat']

mca,train_df_mca,validation_df_mca,test_df_mca=transfo_mca(col_desc_quali,train_df,validation_df,test_df)

Enregistrement sur googlecolab

In [ ]:
train_df_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset_reduit_mca.csv', index=False , encoding='utf-8')
validation_df_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset_reduit_mca.csv', index=False , encoding='utf-8')
test_df_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset_reduit_mca.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
train_df_mca.to_csv('../Data/Datapreprocessing/Complete/Train_Dataset_reduit_mca.csv', index=False , encoding='utf-8')
validation_df_mca.to_csv('../Data/Datapreprocessing/Complete/Validation_Dataset_reduit_mca.csv', index=False , encoding='utf-8')
test_df_mca.to_csv('../Data/Datapreprocessing/Complete/Complete/Test_Dataset_reduit_mca.csv', index=False , encoding='utf-8')

Lecture sur googlecolab

In [ ]:
train_df_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset_reduit_mca.csv', low_memory=False)
validation_df_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset_reduit_mca.csv', low_memory=False)
test_df_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset_reduit_mca.csv', low_memory=False)


Lecture en local

In [ ]:
train_df_mca = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Dataset_reduit_mca.csv', low_memory=False)
validation_df_mca = pd.read_csv('../Data/Datapreprocessing/Complete/Validation_Dataset_reduit_mca.csv', low_memory=False)
test_df_mca = pd.read_csv('../Data/Datapreprocessing/Complete/Test_Dataset_reduit_mca.csv', low_memory=False)

##### Pour jeux réduit

In [ ]:
# Tourne pendant 5 min

col_desc_quali = ['Incident_Type', 'PropertyCategory', 'BoroughCode', 'Station_Code','HCat']

mca1,train_reduit1_mca,val_reduit1_mca,test_reduit1_mca=transfo_mca(col_desc_quali,train_reduit1,val_reduit1,test_reduit1)

mca2,train_reduit2_mca,val_reduit2_mca,test_reduit2_mca=transfo_mca(col_desc_quali,train_reduit2,val_reduit2,test_reduit2)

Enregistrement sur google colab

In [ ]:
train_reduit1_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset_reduit1_mca.csv', index=False , encoding='utf-8')
val_reduit1_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset_reduit1_mca.csv', index=False , encoding='utf-8')
test_reduit1_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset_reduit1_mca.csv', index=False , encoding='utf-8')

train_reduit2_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset_reduit2_mca.csv', index=False , encoding='utf-8')
val_reduit2_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset_reduit2_mca.csv', index=False , encoding='utf-8')
test_reduit2_mca.to_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset_reduit2_mca.csv', index=False , encoding='utf-8')

Enregistrement en local

In [ ]:
train_reduit1_mca.to_csv('../Data/Datapreprocessing/Complete/Train_Dataset_reduit1_mca.csv', index=False , encoding='utf-8')
val_reduit1_mca.to_csv('../Data/Datapreprocessing/Complete/Validation_Dataset_reduit1_mca.csv', index=False , encoding='utf-8')
test_reduit1_mca.to_csv('../Data/Datapreprocessing/Complete/Complete/Test_Dataset_reduit1_mca.csv', index=False , encoding='utf-8')

train_reduit2_mca.to_csv('../Data/Datapreprocessing/Complete/Train_Dataset_reduit2_mca.csv', index=False , encoding='utf-8')
val_reduit2_mca.to_csv('../Data/Datapreprocessing/Complete/Validation_Dataset_reduit2_mca.csv', index=False , encoding='utf-8')
test_reduit2_mca.to_csv('../Data/Datapreprocessing/Complete/Complete/Test_Dataset_reduit2_mca.csv', index=False , encoding='utf-8')

Lecture sur googlecolab

In [ ]:
train_reduit1_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset_reduit1_mca.csv', low_memory=False)
val_reduit1_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset_reduit1_mca.csv', low_memory=False)
test_reduit1_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset_reduit1_mca.csv', low_memory=False)

train_reduit2_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Train_Dataset_reduit2_mca.csv', low_memory=False)
val_reduit2_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Validation_Dataset_reduit2_mca.csv', low_memory=False)
test_reduit2_mca = pd.read_csv('/content/gdrive/My Drive/1_Rendu/FinalDatasets/Complete/Test_Dataset_reduit2_mca.csv', low_memory=False)

Lecture en local

In [ ]:
train_reduit1_mca = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Dataset_reduit1_mca.csv', low_memory=False)
val_reduit1_mca = pd.read_csv('../Data/Datapreprocessing/Complete/Validation_Dataset_reduit1_mca.csv', low_memory=False)
test_reduit1_mca = pd.read_csv('../Data/Datapreprocessing/Complete/Test_Dataset_reduit1_mca.csv', low_memory=False)

train_reduit2_mca = pd.read_csv('../Data/Datapreprocessing/Complete/Train_Dataset_reduit2_mca.csv', low_memory=False)
val_reduit2_mca = pd.read_csv('../Data/Datapreprocessing/Complete/Validation_Dataset_reduit2_mca.csv', low_memory=False)
test_reduit2_mca = pd.read_csv('../Data/Datapreprocessing/Complete/Test_Dataset_reduit2_mca.csv', low_memory=False)